<a href="https://colab.research.google.com/github/Abhishek2104200/QML/blob/main/quantum_kernal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load your full feature dataset
file_path = '/content/drive/MyDrive/balanced_sars_cov2_variants.csv'
df = pd.read_csv(file_path)

print("Dataset shape:", df.shape)
df.head()


Dataset shape: (75000, 30)


/tmp/ipython-input-2-1843703555.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Accession,Organism_Name,GenBank_RefSeq,Submitters,Organization,Org_location,Release_Date,Pangolin,PangoVersions,Surveillance_Sampling,...,Country,USA,Host,Tissue_Specimen_Source,Collection_Date,BioSample,BioProject,GenBank_Title,Variant_Group,WHO_Variant
0,OX474929.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,"Northumbria University, Newcastle upon Tyne, N...",NaN,2023-04-29,B,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,United Kingdom,NaN,Homo sapiens,NaN,2020-03-17,SAMEA7079119,PRJEB37886,Severe acute respiratory syndrome coronavirus ...,B,Alpha
1,MZ185301.1,Severe acute respiratory syndrome coronavirus 2,GenBank,"Howard,D., Batra,D., Cook,P.W., Moser,K., Pask...","Centers for Disease Control and Prevention, Re...",USA,2021-05-13,B.1.526,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,True,...,USA,ME,Homo sapiens,oronasopharynx,2021-04-29,SAMN22689283,PRJNA731148,Severe acute respiratory syndrome coronavirus ...,B,Alpha
2,OU389953.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,London,United Kingdom,2021-07-13,B.1.1.7,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,United Kingdom,NaN,Homo sapiens,NaN,2021-01-11,SAMEA8368730,PRJEB37886,Severe acute respiratory syndrome coronavirus ...,B,Alpha
3,OX597302.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,"Public Health Wales Microbiology Cardiff, Univ...",NaN,2023-05-13,B.1.177.15,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,United Kingdom,NaN,Homo sapiens,NaN,2020-12-01,SAMEA7813988,PRJEB37886,Severe acute respiratory syndrome coronavirus ...,B,Alpha
4,PP603966.1,Severe acute respiratory syndrome coronavirus 2,GenBank,"Amin,H., Wang,J., Su,M., Akther,S., Bologna,J....","NYC DOHMH, Bureau of the Public Health Laboratory",USA,2024-04-09,B.1.1.434,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,USA,NaN,Homo sapiens,NaN,2021-02-10,SAMN20961554,PRJNA704697,Severe acute respiratory syndrome coronavirus ...,B,Alpha


In [ ]:
df.columns

Index(['Accession', 'Organism_Name', 'GenBank_RefSeq', 'Submitters',
       'Organization', 'Org_location', 'Release_Date', 'Pangolin',
       'PangoVersions', 'Surveillance_Sampling', 'Isolate', 'Species', 'Genus',
       'Family', 'Molecule_type', 'Length', 'Nuc_Completeness', 'Genotype',
       'Segment', 'Geo_Location', 'Country', 'USA', 'Host',
       'Tissue_Specimen_Source', 'Collection_Date', 'BioSample', 'BioProject',
       'GenBank_Title', 'Variant_Group', 'WHO_Variant'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



# Drop purely ID-related or non-informative columns
drop_cols = [
    'Accession', 'GenBank_RefSeq', 'Submitters', 'Organization',
    'BioSample', 'BioProject', 'GenBank_Title', 'Isolate'
]
df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Handle date columns: convert to datetime and then to numeric
for col in ['Release_Date', 'Collection_Date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[col] = (df[col] - df[col].min()).dt.days  # Days since earliest date

# Fill missing numerical values with median
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill missing categorical values with 'Unknown'
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna("Unknown")

# Encode categorical features using LabelEncoder (can switch to OneHot for tree models)
encoders = {}
# Convert boolean columns to string (so they can be handled like categories)
for col in cat_cols:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(str)
 # save if you need to inverse-transform later

# Final feature and label separation
X = df.drop(columns=['WHO_Variant'])
y = df['WHO_Variant']
y = LabelEncoder().fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
df.columns


Index(['Organism_Name', 'Org_location', 'Release_Date', 'Pangolin',
       'PangoVersions', 'Surveillance_Sampling', 'Species', 'Genus', 'Family',
       'Molecule_type', 'Length', 'Nuc_Completeness', 'Genotype', 'Segment',
       'Geo_Location', 'Country', 'USA', 'Host', 'Tissue_Specimen_Source',
       'Collection_Date', 'Variant_Group', 'WHO_Variant'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify object-type columns (including those with 'Unknown')
object_cols = X.select_dtypes(include=['object']).columns

# Encode each using LabelEncoder
for col in object_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))


In [ ]:
# Re-split with cleaned X
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [ ]:
import torch

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select exactly 10 features (can be top features or first 10)
X_qml = X.iloc[:, :10].copy()

# Normalize to [0, π] for angle embedding
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_qml = scaler.fit_transform(X_qml)

# Train-test split
Xq_train, Xq_test, yq_train, yq_test = train_test_split(X_qml, y, stratify=y, test_size=0.2, random_state=42)

# Convert to torch tensors
Xq_train_tensor = torch.tensor(Xq_train, dtype=torch.float32)
yq_train_tensor = torch.tensor(yq_train, dtype=torch.long)
Xq_test_tensor = torch.tensor(Xq_test, dtype=torch.float32)
yq_test_tensor = torch.tensor(yq_test, dtype=torch.long)


In [ ]:
!pip install pennylane

# **Quantum Kernal**

In [ ]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = X_qml.shape[1]
dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')
    qml.BasicEntanglerLayers(weights=np.ones((1, n_qubits)), wires=range(n_qubits))  # Dummy weights


In [ ]:
n_qubits = Xq_train.shape[1]
dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')
    qml.BasicEntanglerLayers(weights=np.ones((1, n_qubits)), wires=range(n_qubits))

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits), rotation='Y')
    qml.BasicEntanglerLayers(weights=np.ones((1, n_qubits)), wires=range(n_qubits))
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits), rotation='Y')
    qml.adjoint(qml.BasicEntanglerLayers)(weights=np.ones((1, n_qubits)), wires=range(n_qubits))
    return qml.probs(wires=0)


In [ ]:
def compute_kernel_matrix(X1, X2):
    N1 = len(X1)
    N2 = len(X2)
    kernel = np.zeros((N1, N2))
    for i in range(N1):
        for j in range(N2):
            kernel[i, j] = kernel_circuit(X1[i], X2[j])[0]  # Use prob of |0⟩
    return kernel


In [ ]:
from sklearn.svm import SVC

In [ ]:
Xq_train_np = np.array(Xq_train)
Xq_test_np = np.array(Xq_test)

In [ ]:
train_kernel = compute_kernel_matrix(Xq_train_np, Xq_train_np)
test_kernel = compute_kernel_matrix(Xq_test_np, Xq_train_np)

KeyboardInterrupt: 

In [ ]:
 svc = SVC(kernel='precomputed')
svc.fit(train_kernel, yq_train)
y_pred = svc.predict(test_kernel)


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print("\nClassification Report (QSVM):")
print(classification_report(yq_test, y_pred))
print("Accuracy:", accuracy_score(yq_test, y_pred))


In [ ]:
def compute_kernel_matrix(X1, X2):
    N1 = len(X1)
    N2 = len(X2)
    kernel = np.zeros((N1, N2))
    for i in range(N1):
        for j in range(N2):
            kernel[i, j] = kernel_circuit(X1[i], X2[j])[0]
        print(f"Computed row {i+1}/{N1}")
    return kernel


In [ ]:
Xq_train_np_small = Xq_train_np[:100]
yq_train_small = yq_train[:100]
Xq_test_np_small = Xq_test_np[:30]
yq_test_small = yq_test[:30]


In [ ]:
train_kernel = compute_kernel_matrix(Xq_train_np_small, Xq_train_np_small)
test_kernel = compute_kernel_matrix(Xq_test_np_small, Xq_train_np_small)


Computed row 1/100
Computed row 2/100
Computed row 3/100
Computed row 4/100
Computed row 5/100
Computed row 6/100
Computed row 7/100
Computed row 8/100
Computed row 9/100
Computed row 10/100
Computed row 11/100
Computed row 12/100
Computed row 13/100
Computed row 14/100
Computed row 15/100
Computed row 16/100
Computed row 17/100
Computed row 18/100
Computed row 19/100
Computed row 20/100
Computed row 21/100
Computed row 22/100
Computed row 23/100
Computed row 24/100
Computed row 25/100
Computed row 26/100
Computed row 27/100
Computed row 28/100
Computed row 29/100
Computed row 30/100
Computed row 31/100
Computed row 32/100
Computed row 33/100
Computed row 34/100
Computed row 35/100
Computed row 36/100
Computed row 37/100
Computed row 38/100
Computed row 39/100
Computed row 40/100
Computed row 41/100
Computed row 42/100
Computed row 43/100
Computed row 44/100
Computed row 45/100
Computed row 46/100
Computed row 47/100
Computed row 48/100
Computed row 49/100
Computed row 50/100
Computed 

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel='precomputed')
svc.fit(train_kernel, yq_train_small)

y_pred = svc.predict(test_kernel)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print("\nClassification Report (QSVM):")
print(classification_report(yq_test_small, y_pred))
print("Accuracy:", accuracy_score(yq_test_small, y_pred))



Classification Report (QSVM):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.20      1.00      0.33         6
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         9
           4       0.00      0.00      0.00         4

    accuracy                           0.20        30
   macro avg       0.04      0.20      0.07        30
weighted avg       0.04      0.20      0.07        30

Accuracy: 0.2


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# **QKNN**

In [ ]:
import numpy as np
import pennylane as qml
from collections import Counter
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
import pennylane as qml

n_qubits = Xq_train.shape[1]
dev = qml.device("default.qubit", wires=n_qubits)


In [ ]:
def encode(x):
    qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')
    qml.BasicEntanglerLayers(weights=np.ones((1, n_qubits)), wires=range(n_qubits))

@qml.qnode(dev)
def fidelity_circuit(x1, x2):
    encode(x1)
    qml.adjoint(encode)(x2)
    return qml.probs(wires=0)


In [ ]:
def fidelity(x1, x2):
    return fidelity_circuit(x1, x2)[0]  # Returns probability of |0⟩


In [ ]:
fidelity(Xq_test[0], Xq_train[0])



tensor(1., requires_grad=True)

In [ ]:
print(f"Xq_train shape: {Xq_train.shape}")
print(f"Xq_test shape: {Xq_test.shape}")
print(f"First fidelity: {fidelity(Xq_test[0], Xq_train[0])}")


Xq_train shape: (60000, 10)
Xq_test shape: (15000, 10)
First fidelity: 1.0000000000000013


In [ ]:
from tqdm import tqdm
from collections import Counter

def qknn_predict(X_train, y_train, X_test, k=3):
    preds = []
    for x_test in tqdm(X_test, desc="Classifying with QKNN"):
        sims = []
        for x_train in X_train:
            try:
                sim = fidelity(x_test, x_train)
            except Exception as e:
                print("⚠️ Fidelity error:", e)
                sim = 0.0
            sims.append(sim)

        k_idxs = np.argsort(sims)[-k:]  # indices of top k similarities
        k_labels = y_train[k_idxs]
        most_common = Counter(k_labels).most_common(1)[0][0]
        preds.append(most_common)
    return np.array(preds)


In [ ]:
batch_size = 100
all_preds = []


In [ ]:
for start in range(0, len(Xq_test), batch_size):
    end = start + batch_size
    print(f"🔄 Processing batch {start} to {end}")
    batch_preds = qknn_predict(Xq_train, yq_train, Xq_test[start:end], k=3)
    all_preds.extend(batch_preds)


🔄 Processing batch 0 to 100


Classifying with QKNN:   0%|          | 0/100 [01:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Reduce sample size for speed
Xq_train_small = Xq_train[:100]
yq_train_small = yq_train[:100]
Xq_test_small = Xq_test[:30]
yq_test_small = yq_test[:30]

# Predict
y_pred_qknn = qknn_predict(Xq_train_small, yq_train_small, Xq_test_small, k=3)


Classifying with QKNN: 100%|██████████| 30/30 [01:01<00:00,  2.05s/it]


In [ ]:
print("\nClassification Report (QKNN):")
print(classification_report(yq_test_small, y_pred_qknn))
print("Accuracy:", accuracy_score(yq_test_small, y_pred_qknn))



Classification Report (QKNN):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         6
           2       0.40      0.25      0.31         8
           3       0.71      0.56      0.62         9
           4       0.17      0.25      0.20         4

    accuracy                           0.27        30
   macro avg       0.26      0.21      0.23        30
weighted avg       0.34      0.27      0.30        30

Accuracy: 0.26666666666666666


In [ ]:
import pandas as pd

# Convert to DataFrame for stratified sampling
train_df = pd.DataFrame(Xq_train)
train_df['label'] = yq_train

# Define how many samples per class
samples_per_class = 20
balanced_samples = []

for label in train_df['label'].unique():
    class_subset = train_df[train_df['label'] == label]
    balanced_samples.append(class_subset.sample(n=samples_per_class, random_state=42))

balanced_df = pd.concat(balanced_samples)
Xq_train_balanced = balanced_df.drop(columns='label').values
yq_train_balanced = balanced_df['label'].values
